In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
import pandas as pd


In [ ]:
!pip install wandb



     |████████████████████████████████| 1.7 MB 6.7 MB/s 
     |████████████████████████████████| 181 kB 53.6 MB/s 
     |████████████████████████████████| 144 kB 70.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=3fe627d1c7ed597fea9420e32be7db3f8fe9398e745203456e12638f204afc9d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import wandb
from wandb.keras import WandbCallback
get_ipython().system('wandb login')


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [17]:
wandb.init()
# ... calculate metrics, generate media
wandb.log({"accuracy": 0.9})

wandb: Currently logged in as: lava (use `wandb login --relogin` to force relogin)


In [ ]:
((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

print(x_train.shape, y_train.shape)

x = x_train.reshape(x_train.shape[1]*x_train.shape[2],x_train.shape[0])[:,0:6000]
y_true = y_train.reshape(1,y_train.shape[0])[0:6000]

y_k = y_train[0:6000]
wt = [0,0]

#x_train = np.array([[0,0,0],[0,0,1],[0,1,0],[1,0,0],[1,0,1],[1,1,0],[1,1,1]])
#y_train = np.array([0,1,1,1,0,0,1])

print(x.shape, y_true.shape)



4431872/4422102 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)
(784, 6000) (1, 60000)


In [ ]:
y_hot = np.zeros([len(set(y_k)),len(y_k)])
for i in range(y_hot.shape[1]):
  y_hot[y_k[i]][i] = 1
print(y_hot.shape)

(10, 6000)


In [ ]:
"""
A class to create an object of one layer along with parameters to specifiy:
  input dimensions,
  number of nodes,
  activation function,
  randomMultiplier - Intial scaling factor for weights
"""
class Layer:
    
    def __init__(self, inputDimension, noOfNodes, activation='',optimizer='standard', randomMultiplier=0.01):
        self.weights, self.bias = self.initialize(inputDimension, noOfNodes, randomMultiplier)
        #self.prevMW, self.prevMb = np.zeros([noOfNodes,inputDimension]),np.ones([noOfNodes, 1])
        #self.prevVW, self.prevVb = np.zeros([noOfNodes,inputDimension]),np.ones([noOfNodes, 1])

        # Optimization Alogorithm for gradient Descent
        if optimizer == 'standard':
            self.optimizer = self.standard
        elif optimizer == 'momentumGD':
            self.optimizer = self.momentumGD
        elif optimizer == 'rmsprop':
            self.optimizer = self.rmsprop 
        elif optimizer == 'adam':
            self.optimizer = self.adam
        elif optimizer == 'nadam':
            self.optimizer = self.nadam


        # Activation Function for each layer
        if activation == 'sigmoid':
            self.activation = activation
            self.activationForward = self.sigmoid
            self.activationBackward = self.sigmoidGrad
        else:
            self.activation = 'softmax' 
            self.activationForward = self.softmax
            self.activationBackward = self.softmaxGrad
    
    # Initialize the layer with some random weights and bias
    def initialize(self, noOfInputFeatures, noOfNodes, randomMultiplier):
        np.random.seed(0)
        weights = randomMultiplier * np.random.randn(noOfNodes, noOfInputFeatures)
        bias = np.ones([noOfNodes, 1])
        return weights, bias

    def param_intitialize(self,inputDimension,noOfNodes):
        self.prevMW, self.prevMb = np.zeros([noOfNodes,inputDimension]),np.ones([noOfNodes, 1])
        self.prevVW, self.prevVb = np.zeros([noOfNodes,inputDimension]),np.ones([noOfNodes, 1])

    # Sigmoid activation function
    def sigmoid(self, Z):
        A = 1 / (1 + np.exp(-Z))
        return A

    # Differential of sigmoid function with chain rule applied
    def sigmoidGrad(self, dA):
        s = 1 / (1 + np.exp(-self.prevZ))
        dZ = dA * s * (1 - s)
        return dZ

    #SoftMax Activation Function
    def softmax(self,Z):
      expZ = np.exp(Z - np.max(Z))
      A = expZ / expZ.sum(axis=0, keepdims=True)
      return A

    def softmaxGrad(self,dA):
      #a = np.exp(self.prevZ)
      #A = a / np.sum(a, axis=0, keepdims=True)
      #dZ = dA + A
      #return dZ
      k = self.prevZ
      return dA


    # Take's the input vector A and does the forward pass using weights and bias
    def forward(self, A):
        Z = np.dot(self.weights, A) + self.bias
        self.prevZ = Z
        self.prevA = A
        A = self.activationForward(Z)
        return A
    
    # Take's the input vector derivative of A (dA) and does the backward pass
    def backward(self, dA):
        dZ = self.activationBackward(dA)
        m = self.prevA.shape[1]
        self.dW = 1 / m * np.dot(dZ, self.prevA.T) 
        self.db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        prevdA = np.dot(self.weights.T, dZ)
        return prevdA

    def sgd(self, dA):
        dZ = self.activationBackward(dA)
        prevdA = np.dot(self.weights.T, dZ)
        m = self.prevA.shape[1]
        for i in range(m):
          self.dW = 1 / m * np.dot(dZ[:,i:i+1], self.prevA[:,i:i+1].T)   #arr[:,2]
          self.db = 1 / m * (dZ[:,i:i+1])
          self.weights = self.weights - (0.02 * self.dW)
          self.bias = self.bias - (0.02 * self.db)
        #prevdA = np.dot(self.weights.T, dZ)
        return prevdA
    
    # Update's the weights and bias with the passed learning_rate
    def standard(self, learning_rate,t):
        self.weights = self.weights - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db
    
    def momentumGD(self, learning_rate,t, moving_rate=0.9):
        self.prevMW = moving_rate * self.prevMW + (1-moving_rate) * self.dW
        self.prevMb = moving_rate * self.prevMb + (1-moving_rate) * self.db
        self.weights = self.weights - learning_rate * self.prevMW
        self.bias = self.bias - learning_rate * self.prevMb
        

    def rmsprop(self, learning_rate,t, moving_rate = 0.9):
        self.prevVW = moving_rate * self.prevVW + (1-moving_rate) * np.square(self.dW)
        self.prevVb = moving_rate * self.prevVb + (1-moving_rate) * np.square(self.db)
        self.weights = self.weights - learning_rate * np.divide(self.dW,np.sqrt(self.prevVW+(1e-8)))
        self.bias = self.bias - learning_rate * np.divide(self.db,np.sqrt(self.prevVb+(1e-8)))


    def adam(self,learning_rate ,t, beta1 = 0.9, beta2 = 0.999):
        self.prevMW = beta1 * self.prevMW + (1-beta1)*self.dW
        self.prevMb = beta1 * self.prevMb + (1-beta1)*self.db

        self.prevVW = beta2 * self.prevVW + (1-beta2)*np.square(self.dW)
        self.prevVb = beta2 * self.prevVb + (1-beta2)*np.square(self.db)
        
        self.prevMW = self.prevMW/(1-beta1**(t+1))
        self.prevMb = self.prevMb/(1-beta1**(t+1))

        self.prevVW = self.prevVW/(1-beta2**(t+1))
        self.prevVb = self.prevVb/(1-beta2**(t+1))

        self.weights = self.weights - learning_rate * np.divide(self.prevMW,np.sqrt(self.prevVW+(1e-8)))
        self.bias = self.bias - learning_rate * np.divide(self.prevMb,np.sqrt(self.prevVb+(1e-8)))
        """
        MWhat = self.prevMW/(1-beta1**(t+1))
        Mbhat = self.prevMb/(1-beta1**(t+1))

        VWhat = self.prevVW/(1-beta2**(t+1))
        Vbhat = self.prevVb/(1-beta2**(t+1))

        self.weights = self.weights - learning_rate * np.divide(MWhat,np.sqrt(VWhat+(1e-8)))
        self.bias = self.bias - learning_rate * np.divide(Mbhat,np.sqrt(Vbhat+(1e-8)))
        """


    def nadam(self,learning_rate ,t, beta1 = 0.9, beta2 = 0.999):
        self.prevMW = beta1 * self.prevMW + (1-beta1)*self.dW
        self.prevMb = beta1 * self.prevMb + (1-beta1)*self.db

        self.prevVW = beta2 * self.prevVW + (1-beta2)*np.square(self.dW)
        self.prevVb = beta2 * self.prevVb + (1-beta2)*np.square(self.db)

        MWhat = (beta1 * self.prevMW / (1-beta1)) + (self.dW)   #(b1 * m[t] / (1.0 - b1)) + ((1 - b1) * g(t) / (1.0 - b1))
        Mbhat = (beta1 * self.prevMb / (1-beta1)) + (self.db)

        VWhat = (beta2 * self.prevVW) / (1-beta2)
        Vbhat = (beta2 * self.prevVb) / (1-beta2)

        self.weights = self.weights - learning_rate * np.divide(MWhat,np.sqrt(VWhat+(1e-8)))
        self.bias = self.bias - learning_rate * np.divide(Mbhat,np.sqrt(Vbhat+(1e-8)))

        #self.prevMW = (beta1 * self.prevMW / (1-beta1)) + (self.dW)
        #self.prevMb = (beta1 * self.prevMb / (1-beta1)) + (self.db)

        #self.prevVW = (beta2 * self.prevVW) / (1-beta2)
        #self.prevVb = (beta2 * self.prevVb) / (1-beta2)

        #self.weights = self.weights - learning_rate * np.divide(self.prevMW,np.sqrt(self.prevVW+(1e-8)))
        #self.bias = self.bias - learning_rate * np.divide(self.prevMb,np.sqrt(self.prevVb+(1e-8)))
        
          
        



In [ ]:
"""
Neural Network which consists of all layers and some helper functions
"""
class NeuralNetwork:
    
    def __init__(self, loss, optimizer = 'standard', randomMultiplier = 0.01):
        self.layers=[]
        self.optimizer = optimizer
        self.randomMultiplier = randomMultiplier
        if loss=='cross_entropy':
            self.lossFunction = self.cross_entropy
            self.lossBackward = self.cross_entropy_Grad
        elif loss == 'mean_square':
            self.lossFunction = self.meanSquareError
            self.lossBackward = self.meanSquareErrorGrad
        else:
            print('Invalid loss function')
        self.loss=loss

    # add layer to the NN with input dimensions, nodes & activation function
    def addLayer(self, inputDimension=None, noOfNodes=1, activation=''):
        if (inputDimension is None):
            if (len(self.layers) == 0):
                print('Invalid number of layers')
            inputDimension = self.layers[-1].outputDimension()
        layer = Layer(inputDimension, noOfNodes, activation,optimizer = self.optimizer, randomMultiplier=self.randomMultiplier)
        self.layers.append(layer)

    def meanSquareError(self, Y, A):
        loss = np.square(Y - A)
        m = Y.shape[1]
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
    
    # Compute's mean square grad error, Y is true value & A is predicted value
    def meanSquareErrorGrad(self, Y, A):
        dA = -2 * (Y - A)
        return dA

    # Compute's mean square error where Y is true value and A is predicted value
    def cross_entropy(self, Y, A):
        m = Y.shape[1]
        cost = -(1/m)*np.sum(Y*np.log(A))
        return np.squeeze(cost)
    
    # Compute's mean square grad error, Y is true value & A is predicted value
    def cross_entropy_Grad(self, Y, A):
        dA = A-Y
        return dA

    # Wrapper function to get the cost or loss value of the predicted values
    def cost(self, Y, A):
        return self.lossFunction(Y, A)

    # Forward pass the input vector X through all layers
    def forward(self, X):
        x = np.copy(X)
        for layer in self.layers:
            x = layer.forward(x)
        return x
            
    # Backward pass the true values and predicted values in reverse direction
    def backward(self, Y, A):
        dA = self.lossBackward(Y, A)
        if(self.optimizer != 'sgd'):
            for layer in reversed(self.layers):
                dA = layer.backward(dA)
        else:
          for layer in reversed(self.layers):
                dA = layer.sgd(dA)
    
    # Update weights and compute's gradient descent of all layers
    def update(self, learning_rate=0.02,t=0):
        for layer in self.layers:
            layer.optimizer(learning_rate,t=0)


    def fit(self,x,y):
        layers=[]
        x = x.reshape(x.shape[0],x.shape[1]*x.shape[2]).T
        layers.append(x.shape[0])

        # Adding hidden layer with sigmoid as activation function
        hidden_layers = int(input("Enter number of Hidden Layers : "))
        for i in range(hidden_layers):
          tmp = int(input("Enter number of nodes you want in Hidden Layer "+str(i+1)+" : "))
          layers.append(tmp)
          self.addLayer(inputDimension=layers[i], noOfNodes=tmp, activation='sigmoid')

        y_hot = np.zeros([len(set(y)),len(y)])

        for i in range(y_hot.shape[1]):
          y_hot[y[i]][i] = 1
          

        # Output layer just to collect output (not a hidden layer)
        self.addLayer(inputDimension=layers[hidden_layers], noOfNodes=y_hot.shape[0], activation='softmax')
        for i in range(10):
          A = self.forward(x)
          self.backward(y_hot,A)
          if(self.optimizer != 'sgd'):
            self.update(learning_rate=0.01,t= i+1)
 
          if not i%2 or ((y == y_pred).mean()*100>96):
            print("After ",i+1,"iterations:")
            print('cross Entropy:', model.cost(y_hot, A)) #2.3029423349010876
            y_pred = np.argmax(A,axis = 0)
            print('Training Accuracy:',(y == y_pred).mean()*100)

            #for j in range(10):
             # print(j,"=>",list(y_pred).count(j))


            #break the loop when model is going to converge (accuracy: atleast 96%)

            if ((y == y_pred).mean()*100>96):
              #A = self.forward(x_test.reshape(x_test.shape[1]*x_test.shape[2],x_test.shape[0])[0:100])
              #return A
              break

    def predict(self,x):
        x = x.reshape(x.shape[0],x.shape[1]*x.shape[2]).T
        A = self.forward(x)
        A = np.argmax(A,axis=0)
        return A


In [ ]:
np.random.seed(1)
model = NeuralNetwork(loss='cross_entropy', optimizer = 'standard')
y_pred = model.fit(x_train,y_train)

Enter number of Hidden Layers : 0
After  1 iterations:
cross Entropy: 47.37754644544284
Training Accuracy: 7.673333333333334


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide


After  3 iterations:
cross Entropy: nan
Training Accuracy: 10.0
After  5 iterations:
cross Entropy: nan
Training Accuracy: 10.0
After  7 iterations:
cross Entropy: nan
Training Accuracy: 10.0
After  9 iterations:
cross Entropy: nan
Training Accuracy: 10.0


In [ ]:
y_p = model.predict(x_test)
print((y_p == y_test).mean()*100)
print(y_p)

10.0
[0 0 0 ... 0 0 0]


In [ ]:
y_p = model.predict(x_train[0:100])
y_pr = np.argmax(y_p,axis = 0)
print((y_pr == y_train[0:100]).mean()*100)
print(y_pr)

12.0
0
